# fcc and bcc Crystals

Structure:
1. create fcc unit cell
2. create bcc unit cell
3. create supercell
4. sanity check neighbour distances

TODOs:
* sort out `box`, `box_size`, `box_vectors`
* export code to dedicated crystal module / jl files

In [ ]:
import Pkg

In [ ]:
Pkg.activate(".")

In [ ]:
Pkg.status()

In [ ]:
using DataFrames
using Molly
using Plots
using Test
using LaTeXStrings
using LinearAlgebra
using SparseArrays

## Creating Crystals

In [ ]:
masses = Dict("V" => 50.9415, "Nb" => 92.9064, "Ta" => 180.9479,
              "Cr" => 51.996, "Mo" => 95.94, "W" => 183.85,
              "Fe" => 55.847)

### An atom

In [ ]:
element = "Fe"
Atom(name=element, mass=masses[element])

## Synthesizing crystal unit cells

Crystals are something fascinating. Defect free crystals are highly symmetric and can be reduced to so-called "unit cells", a cell which can be used by copying and shifting it to construct the entire crystal. So to sound impressive the crystal of multiple unit cells is called a supercell ¯\\\_(ツ)\_/¯.

So in a first step we'll define how to create two common types of unit cells and then go on to synthesize a supercell.

In [ ]:
# Å
bcc_lattice_constants = Dict(
    "V" => 3.0399, "Nb" => 3.3008, 
    "Ta" => 3.3058, "Cr" => 2.8845, "Mo" => 3.1472, 
    "W" => 3.1652, "Fe" => 2.8665
)

### face centered cubic crystal

In [ ]:
function make_fcc_unitcell(element::String;a::T=1) where T <:Real
    coords = [[0 0 0],[1//2 1//2 0],
        [1//2 0 1//2],[0 1//2 1//2]]
    atoms = [Atom(name=element, mass=masses[element]) 
             for _ in coords]
    box_size = Diagonal([a, a, a])
    box_vectors = [1. 0. 0.; 0. 1. 0.; 0. 0. 1.]
    box = box_vectors * box_size
    coords = [v*box for v in coords]
    return atoms, coords, box, box_size, box_vectors
end

In [ ]:
element = "Fe"
a = bcc_lattice_constants[element]
atoms, coords, box, box_size, box_vectors = make_fcc_unitcell(element, a=a)

In [ ]:
function plot_crystal(atoms::Array, coords::Array; 
        default_color::String="blue",
        element_color_map::Dict=Dict{String,String}(),
        default_size::T=50,
        element_size_map::Dict=Dict{String,Any}()
    ) where T <: Real
    
    elements = Set([atom.name for atom in atoms])
    for element in elements
        if !haskey(element_color_map, element)
            element_color_map[element] = default_color
        end
        if !haskey(element_size_map, element)
            element_size_map[element] = default_size
        end
    end
    colors = [element_color_map[element] for element in elements]
    sizes = [element_size_map[element] for element in elements]

    x = [v[1] for v in coords]
    y = [v[2] for v in coords]
    z = [v[3] for v in coords]
    return @gif for i in range(0, stop=2π, length=100)
        scatter(x, y, z, camera=(10*(1+cos(i)),5),
            markersize=sizes, legend=false, 
            color=colors, aspect_ratio=:equal,
            xlabel=L"x", ylabel=L"y", zlabel=L"z",
            title=string(length(atoms), " atoms of: ", join(elements, ","))
        )
    end
end

In [ ]:
plot_crystal(atoms, coords, default_color="red", default_size=50)

### body centered cubic crystal

In [ ]:
function make_bcc_unitcell(element::String;a::T=1) where T <:Real
    coords =[[0 0 0], [.5 .5 .5]]
    atoms = [Atom(name=element, mass=masses[element]) 
             for _ in coords]
    box_size = Diagonal([a, a, a])
    box_vectors = [1. 0. 0.; 0. 1. 0.; 0. 0. 1.]
    box = box_vectors * box_size
    coords = [v*box for v in coords]
    return atoms, coords, box, box_size, box_vectors
end

In [ ]:
element = "Fe"
a = bcc_lattice_constants[element]
atoms, coords, box, box_size, box_vectors = make_bcc_unitcell(element, a=a)

In [ ]:
plot_crystal(atoms, coords, default_size=50)

In [ ]:
element = "Fe"
a = bcc_lattice_constants[element]
atoms, coords, box, box_size, box_vectors = make_fcc_unitcell(element, a=a)

## Generating a supercell from a unit cell

In [ ]:
function make_supercell(atoms::Array, coords::Array, 
        box::Array, box_size::Diagonal; nx::Int64=1, ny::Int64=1,
        nz::Int64=1)
    @assert (nx > 0) & (ny > 0) & (nz > 0) 
    sc_atoms = []
    sc_coords = []
    sc_box = box
    sc_box_size = box_size * Diagonal([nx, ny, nz])
    n_atoms = length(coords)
    for i in 0:nx-1, j in 0:ny-1, k in 0:nz-1
        push!(sc_atoms,atoms)
        scale = Diagonal([i,j,k])
        shift = sum(sc_box*scale, dims=1)
        push!(sc_coords,[coords[l]+shift for l in 1:n_atoms])
    end
    sc_atoms = vcat(sc_atoms...)
    sc_coords = vcat(sc_coords...)
    return sc_atoms, sc_coords, sc_box, sc_box_size
end

In [ ]:
sc_atoms, sc_coords, sc_box, sc_box_size = make_supercell(atoms, coords, box, box_size, nx=3, ny=3,
        nz=3);

In [ ]:
plot_crystal(sc_atoms, sc_coords, default_size=10)

In [ ]:
@assert length(sc_atoms) == length(sc_coords)

In [ ]:
n_atoms = length(sc_atoms)

Looks okay so far, let's move on.

## Sanity checking distances

Let's define some minimal objects (`MinimalSimulationConfig`) so we can perform neighbour search in a similar way as is actually done for simulations, but without needing to define interactions.

In [ ]:
mutable struct MinimalSimulationConfig
    atoms::Array
    box_size::Float32
    coords::Array
    neighbours::Array{Tuple{Int64,Int64}}
end

In [ ]:
element = "Fe"
a = 1 #bcc_lattice_constants[element]
atoms, coords, box, box_size, box_vectors = make_fcc_unitcell(element, a=a)
sc_atoms, sc_coords, sc_box, sc_box_size = make_supercell(atoms, coords, box, box_size, nx=3, ny=3,nz=3)
n_atoms = length(sc_atoms);

In [ ]:
initial_neighbours = Tuple{Int,Int}[]
s = MinimalSimulationConfig(sc_atoms, sc_box_size[1,1], sc_coords, initial_neighbours);

For each atom we need to know all its neighbours. 

In [ ]:
struct MyNeighbourFinder <: NeighbourFinder
    nb_matrix::BitArray{2} # defines which atom pairs we'll be happy to check at all
    n_steps::Int
    dist_cutoff::Float32
    rcut2::Float32
end

MyNeighbourFinder(nb_matrix, n_steps, dist_cutoff) = MyNeighbourFinder(nb_matrix, n_steps, dist_cutoff, dist_cutoff^2)

In [ ]:
nb_matrix = trues(n_atoms,n_atoms)
n_steps = 1
dist_cutoff = 2 #*lattice_constants[element]

In [ ]:
nf = MyNeighbourFinder(nb_matrix, n_steps, dist_cutoff)

The basic neighbourhood search algorithm, returning the neighbour coords without modifying `s`

In [ ]:
function simple_find_neighbours(s::MinimalSimulationConfig,
        nf::MyNeighbourFinder, step_n::Int;
        parallel::Bool=false, 
        x_shifts=[0], y_shifts=[0], z_shifts=[0] # factors by which the box will be shifted along each box vector
    )
    
    !iszero(step_n % nf.n_steps) && return
    neighbours = empty(s.neighbours)
    for i in 1:length(s.coords)
        ci = s.coords[i]
        for j in 1:length(s.coords)
            if i==j 
                continue
            end
            
            r2 = sum(abs2, vector(ci, s.coords[j], s.box_size))
            if r2 <= nf.rcut2 && nf.nb_matrix[j,i]
                push!(neighbours, (i,j))
            end                
        end
    end
    return neighbours
end

### fcc

In [ ]:
idxs = simple_find_neighbours(s, nf, 1);

In [ ]:
rs = [sqrt(sum(abs2, vector(s.coords[i], s.coords[j], s.box_size)))
    for (i,j) in idxs
];

In [ ]:
rs_df = sort(combine(groupby(DataFrame("distances"=>rs),[:distances]), nrow=>:count), [:distances])

In [ ]:
@assert rs_df.distances[1] ≈ sqrt(1^2+1^2)/2
@assert rs_df.distances[2] ≈ 1
@assert rs_df.distances[3] ≈ sqrt(1^2+(sqrt(2)/2)^2)
@assert rs_df.distances[4] ≈ sqrt(1^2+1^2)
@assert rs_df.distances[5] ≈ sqrt(3^2+1^2)/2

In [ ]:
histogram(rs, xlabel=L"r", ylabel="Frequency", 
    title=string("FCC: euclidan (periodic) distance distribution (rcut ",nf.dist_cutoff,")"),
    bins=200,
)

### bcc

The same as above but for bcc

In [ ]:
element = "Fe"
a = 1 #bcc_lattice_constants[element]
atoms, coords, box, box_size, box_vectors = make_bcc_unitcell(element, a=a)
sc_atoms, sc_coords, sc_box, sc_box_size = make_supercell(atoms, coords, box, box_size, nx=3, ny=3,nz=3)
n_atoms = length(sc_atoms);

In [ ]:
initial_neighbours = Tuple{Int,Int}[]
s = MinimalSimulationConfig(sc_atoms, sc_box_size[1,1], sc_coords, initial_neighbours);

In [ ]:
nb_matrix = trues(n_atoms,n_atoms)
n_steps = 1
dist_cutoff = 2 #*lattice_constants[element]

In [ ]:
nf = MyNeighbourFinder(nb_matrix, n_steps, dist_cutoff)

In [ ]:
idxs = simple_find_neighbours(s, nf, 1);

In [ ]:
rs = [sqrt(sum(abs2, vector(s.coords[i], s.coords[j], s.box_size)))
    for (i,j) in idxs
];

In [ ]:
rs_df = sort(combine(groupby(DataFrame("distances"=>rs),[:distances]), nrow=>:count), [:distances])

In [ ]:
@assert rs_df.distances[1] ≈ sqrt((sqrt(2)/2)^2 + 1/2^2)
@assert rs_df.distances[2] ≈ 1
@assert rs_df.distances[3] ≈ sqrt(2)
@assert rs_df.distances[4] ≈ sqrt((sqrt(2)/2)^2 + (3/2)^2)
@assert rs_df.distances[5] ≈ sqrt(sqrt(2)^2 + 1^2)

In [ ]:
histogram(rs, xlabel=L"r", ylabel="Frequency", 
    title=string("BCC: euclidan (periodic) distance distribution (rcut ",nf.dist_cutoff,")"),
    bins=200,
)